In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk 
import re
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

In [6]:
df= pd.read_excel('./859181_INDICES_v_3_conductas.xlsx', header= 4)

In [7]:
df ['Fecha']=pd.to_datetime(df['Fecha'])
df.head()

,Unnamed: 0,SP,Fecha,Resol. No.,Empresas,Mercado,Categoria,Conducta,Conducta categorizadas,Sancion,DETALLE,Unnamed: 11
0,NaN,326599,2002-10-17,46,"Beverage Associates Corp, Quilmes Industrialm ...",Mercado de la cerveza,bebidas,Carteles - Acuerdo de Precios - Archivo,Cartelizacion,Archivo,Realización de una operación de concentración ...,NaN
1,NaN,323089,1980-11-09,[127],The Buenos Aires Herald Ltda. / Sociedad de Di...,Distribución de diarios.,comunicacion,Distribución - Negativa -,NaN,Archivo,_,NaN
2,NaN,321373,1981-04-12,124,Acindar Industria Argentina de Aceros S.A. / S...,MANUFACTURING Fabricación de metales comunes -...,energia,Póliticas de precios - Guerra de precios - Con...,NaN,Archivo,Para que la guerra de precios pueda considerar...,NaN
3,NaN,321185,1981-08-09,257,Juan Ignacio Borchex y Oscar Dos Reis / Asocia...,Servicio funerario,funerario,Abuso de posición dominante,Abuso posicion dominante,Archivo,Extensión de servicio mutual a personas no mut...,NaN
4,NaN,321190,1981-08-18,302,Tiboni y Cía S.R.L. / Sorensen y Cía. S.R.L.,Pantógrafos (calado o perforado de cristales p...,automotor,Trato exclusivo - patentes,NaN,Archivo,La denunciada ejerció una serie de presiones s...,NaN


In [8]:
#convertir todas las que son str lowercase
df = df.applymap(lambda s:s.lower() if type(s) == str else s)

#eliminar los simbolos
df['Mercado'] = [re.sub("[\W+]",' ', str(x)) for x in df['Mercado']]
df['Conducta']= [re.sub("[\W+]",' ', str(x)) for x in df['Conducta']]

##convertir esto en una lista de palabras a eliminar.
filtrar = ['mercado', 'servicio', 'servicios', 'fabricación', 'fabricacion', 'fabricacion', 'manufacturing', 'mencionado', 'productos']
df['Mercado'] = df['Mercado'].apply(lambda x: ' '.join([word for word in x.split() if word not in (filtrar)]))
filtrar1 = ['archivo', 'nan'] 
df ['Conducta'] = df ['Conducta'].apply(lambda x: ' '.join([word for word in x.split() if word not in (filtrar1)]))

#eliminamos las stopwords
stopwords = nltk.corpus.stopwords.words('spanish')
df['Mercado'] = df['Mercado'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
df['Conducta'] = df['Conducta'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

##Tareas: en donde es nan el mercado, buscar completar. 
df.replace('', np.nan, inplace=True)

In [25]:
df_energia = df [df["Categoria"] == "energia "].copy()
df_energia.reset_index(drop=True, inplace=True)
df_energia.shape

(100, 12)

In [27]:
# Select observations between two datetimes
df_5= df_energia[(df_energia['Fecha'] > '2015-1-1 01:00:00') & (df_energia['Fecha'] <= '2020-1-1 04:00:00')]

In [28]:
#eliminar los simbolos
df_5['Resol. No.'] = [re.sub("[\D+]",'', str(x)) for x in df_5['Resol. No.']]
lista= df_5 ['Resol. No.'].unique()
lista

<ipython-input-28-e00e4e52c3bc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_5['Resol. No.'] = [re.sub("[\D+]",'', str(x)) for x in df_5['Resol. No.']]


array(['423', '202', '126', '188', '277', '389', '654', '658', '923',
       '740', '885', '16', '26', '214', '31', '77', '653', '604', '627'],
      dtype=object)